In [1]:
from diffusers.models.transformer_2d import Transformer2DModel
from diffusers.models.unet_2d_blocks import UNetMidBlock2DCrossAttn


import torch
import torch.nn as nn
from diffusers.models.resnet import Downsample2D, FirDownsample2D, FirUpsample2D, KDownsample2D, KUpsample2D, ResnetBlock2D, Upsample2D

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from diffusers.models.attention_processor import Attention

In [91]:
64*8

512

In [92]:
num_attention_heads = 4
attention_head_dim = 16
inner_dim = num_attention_heads * attention_head_dim

feature = torch.randn(3, 64, 8, 8)
fr_emb = torch.randn(3, 64, 64)

a = Attention(64, cross_attention_dim=64, dim_head=32)

In [93]:
a

Attention(
  (to_q): Linear(in_features=64, out_features=256, bias=False)
  (to_k): Linear(in_features=64, out_features=256, bias=False)
  (to_v): Linear(in_features=64, out_features=256, bias=False)
  (to_out): ModuleList(
    (0): Linear(in_features=256, out_features=64, bias=True)
    (1): Dropout(p=0.0, inplace=False)
  )
)

In [84]:
a.processor

In [96]:
a(feature, encoder_hidden_states=fr_emb)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (768x64 and 256x64)

In [70]:
# 보니깐 
import torch.nn.functional as F

In [71]:
torch.randn(3, 5, 4, 2).view(3, 5, 4*2).transpose(1, 2).shape

torch.Size([3, 8, 5])

In [ ]:
class AttnProcessor2_0:
    # scaled dot-product attention 수행하는 부분
    def __init__(self):
        if not hasattr(F, "scaled_dot_product_attention"):
            raise ImportError("AttnProcessor2_0 requires PyTorch 2.0, to use it, please upgrade PyTorch to 2.0.")

    def __call__(
        self,
        attn:Attention,
        hidden_states,
        encoder_hidden_states=None,
        attention_mask=None,
        temb=None,
    ):
        residual = hidden_states

        # spatial norm이 있다면=> norm 먼저
        if attn.spatial_norm is not None:
            hidden_states = attn.spatial_norm(hidden_states, temb)


        # 여기서 feature가 3차원인지 4차원인지 구분
        input_ndim = hidden_states.ndim

        # 4차원 > B, HxW, C = B, len, Dim
        if input_ndim == 4:
            batch_size, channel, height, width = hidden_states.shape
            hidden_states = hidden_states.view(batch_size, channel, height * width).transpose(1, 2)

        # 만약 encoder 쪽에서 오는게 없으면 hidden_stats 정보로만 => 아마 selfattnetion 상각인듯
        
        batch_size, sequence_length, _ = (
            hidden_states.shape if encoder_hidden_states is None else encoder_hidden_states.shape
        )

        # 여기서 다시 innder_dim을 설정하면 앞에선 왜한거지?
        inner_dim = hidden_states.shape[-1]

        # attention mask 처리
        if attention_mask is not None:
            attention_mask = attn.prepare_attention_mask(attention_mask, sequence_length, batch_size)
            # scaled_dot_product_attention expects attention_mask shape to be
            # (batch, heads, source_length, target_length)
            attention_mask = attention_mask.view(batch_size, attn.heads, -1, attention_mask.shape[-1])

        if attn.group_norm is not None:
            hidden_states = attn.group_norm(hidden_states.transpose(1,2)).transpose(1, 2)

        # norm 이후에 hiddne
        query = attn.to_q(hidden_states)

        # 만약 encoder에서 오는게 없으면 self attn처럼
        if encoder_hidden_states is None:
            encoder_hidden_states = hidden_states
        # encoder에서 오는게 있으면 => 이것도 일단 norm
        elif attn.norm_cross:
            encoder_hidden_states = attn.norm_encoder_hidden_states(encoder_hidden_states)

        # key
        # only_cross_attention이라면 None일텐데
        key = attn.to_k(encoder_hidden_states)
        value = attn.to_v(encoder_hidden_states)

        # 일단 q,k,v 다 걸고나서 head로 자르는 형태임
        head_dim = inner_dim // attn.heads

        # [B, H, seq, Dim]으로 만들기
        query = query.view(batch_size, -1, attn.heads, head_dim).transpose(1, 2)
        key = key.view(batch_size, -1, attn.heads, head_dim).transpose(1, 2)
        value = value.view(batch_size, -1, attn.heads, head_dim).transpose(1, 2)

        # 실제 attention 일어나는 부분
        hidden_states = F.scaled_dot_product_attention(
            query, key, value, attn_mask=attention_mask, dropout_p=0.0, is_causal=False
        )

        # [B, seq ,H ,DIm ] => [B, seq ,H*DIm ] : [B, 8*192]
        hidden_states = hidden_states.transpose(1, 2).reshape(batch_size, -1, attn.heads * head_dim)
        hidden_states = hidden_states.to(query.dtype)

        # linear proj
        hidden_states = attn.to_out[0](hidden_states) # 원본 query dim으로 
        # dropout
        hidden_states = attn.to_out[1](hidden_states)

        # 원래 4차원 인풋이라면 복구시킴
        if input_ndim == 4:
            hidden_states = hidden_states.transpose(-1, -2).reshape(batch_size, channel, height, width)

        # residual connectin
        if attn.residual_connection:
            hidden_states = hidden_states + residual

        hidden_states = hidden_states / attn.rescale_output_factor

        return hidden_states

In [ ]:
class CrossAttention(nn.Module):
    # 기본적으로 [B, C, DIM] 이런식으로 날아와야함
    # 내부 feature를 Q
    # 외부 condition이 K, V
    def __init__(self):
        super().__init__
        query_dim = 32
        dim_head: int = 64
        heads = 8
        dropout: float = 0.0
        bias=False # 왠만해선 False임. to_k 등에서 bias 넣을지 말지 결정
        
        inner_dim = dim_head * heads
        # 이걸 따로 명시하면 아니면 query_dim임
        # query dim이란?
        # Q로 이용되는 feature의 dim을 의미 => 이를 inner_dim으로 변경할 것임
        
        cross_attention_dim = cross_attention_dim if cross_attention_dim is not None else query_dim

        # 이게 제대로 되렴녀 added_kv_proj_dim이 필요함
        only_cross_attention = False
        added_kv_proj_dim = ...

        if self.added_kv_proj_dim is None and self.only_cross_attention:
            raise ValueError("")




        self.to_q = nn.Linear(query_dim, inner_dim, bias=bias)
        
        # 만약 only cross attention만 쓰는게 아니라면
        # 만약 only_cross_attention=False이면 k,v를 만듬
        if not only_cross_attention:
            self.to_k = nn.Linear(cross_attention_dim, inner_dim, bias=bias)
            self.to_v = nn.Linear(cross_attention_dim, inner_dim, bias=bias)
        else: # 이거만 아예 없음
            self.to_k = None
            self.to_v = None

        #이건 아마 projection 이후에 하는거 같은데?
        # cross attention만 한다면 반드시 필요
        if self.added_kv_proj_dim is not None:
            self.add_k_proj = nn.Linear(added_kv_proj_dim, inner_dim)
            self.add_v_proj = nn.Linear(added_kv_proj_dim, inner_dim)


        # 이부분은 readout
        self.to_out = nn.ModuleList([])
        self.to_out.append(nn.Linear(inner_dim, query_dim, bias=out_bias))
        self.to_out.append(nn.Dropout(dropout))

        # 이부분은 내부적으로 어떤 Attention 라이브러리 쓸지
        # 2.0이 있고 sacle_qk=True이면 2.0 버전
        # 아니면 구버전
        # AttnProcessor2_0에선 add_k_proj, add_v_proj를 처리하지 않음 
        # AttnAddedKVProcessor2_0에 있음
        # 아마 set_use_memory_efficient_attention_xformers이게 실행되면서 설정하는 듯
        if processor is None:
            processor = (
                AttnProcessor2_0() if hasattr(F, "scaled_dot_product_attention") and self.scale_qk else AttnProcessor()
            )
        self.set_processor(processor)


    # 실제 처리하는 부분은 processor가
    def forward(self, hidden_states,
                encoder_hidden_states=None,
                attention_mask=None,
                **cross_attention_kwargs
                ):
        # Attention 자기 자신을 넘겨줌
        return self.processor(
            self,
            hidden_states,
            encoder_hidden_states=encoder_hidden_states,
            attention_mask=attention_mask,
            **cross_attention_kwargs,
        )

In [ ]:
class Attention(nn.Module):
    def __init__(
        self,
        query_dim: int,
        cross_attention_dim: Optional[int] = None,
        heads: int = 8,
        dim_head: int = 64,
        dropout: float = 0.0,
        bias=False,
        upcast_attention: bool = False,
        upcast_softmax: bool = False,
        cross_attention_norm: Optional[str] = None,
        cross_attention_norm_num_groups: int = 32,
        added_kv_proj_dim: Optional[int] = None,
        norm_num_groups: Optional[int] = None,
        spatial_norm_dim: Optional[int] = None,
        out_bias: bool = True,
        scale_qk: bool = True,
        only_cross_attention: bool = False,
        eps: float = 1e-5,
        rescale_output_factor: float = 1.0,
        residual_connection: bool = False,
        _from_deprecated_attn_block=False,
        processor: Optional["AttnProcessor"] = None,
    ):
        super().__init__()
        inner_dim = dim_head * heads # head와 각 head의 dim

In [69]:
a(feature, encoder_hidden_states=fr_emb)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1536x64 and 512x64)

In [38]:


feature = torch.randn(3, 64, 16, 16)
fr_emb = torch.randn(3, 64, 32, 32)

tf_blk = Transformer2DModel(in_channels=64, out_channels=128)

out = tf_blk(feature, fr_emb, return_dict=False,)[0]


print(out.shape)


torch.Size([3, 64, 16, 16])


In [49]:
num_attention_heads=16
in_channels=64
resnet_groups: int = 32
cross_attention_dim=1280
only_cross_attention=True

tf_blk = Transformer2DModel(
                num_attention_heads,
                in_channels // num_attention_heads,
                in_channels = in_channels,
                out_channels=128,
                num_layers=1,
                cross_attention_dim=64,
                only_cross_attention=True)
                # cross_attention_dim=cross_attention_dim)

In [53]:
out = tf_blk(feature, fr_emb, return_dict=False,)[0]


ValueError: too many values to unpack (expected 3)

In [ ]:
class Basic

In [14]:
# (hidden_states = torch.randn(3, 64, 32, 32),
#                                                      encoder_hidden_states=torch.randn(3, 10, 64)).sample.shape


# 일단 feature이던지 encoder_hidden이던지 채널수는 맞아야 함
# 여기선 원래 text embedding을 받지만 SIDM을 위해선 이미지 임베딩을 받아도 될듯
# 둘 다 받아야 함
# time와 image embedding 둘다 받을 수 있게
# 기본적으로 ResNet2D가 time을 받을 수 있음
class UNetMidBlock2DCrossAttnImageKV(nn.Module):
    def __init__(
        self,
        in_channels: int,
        temb_channels: int,
        dropout: float = 0.0,
        num_layers: int = 1,
        resnet_eps: float = 1e-6,
        resnet_time_scale_shift: str = "default",
        resnet_act_fn: str = "swish",
        resnet_groups: int = 32,
        resnet_pre_norm: bool = True,
        num_attention_heads=1,
        add_attention: bool = True,
        output_scale_factor=1.0,
        cross_attention_dim=1280,
    ):
        super().__init__()
        self.has_cross_attention = True
        self.num_attention_heads = num_attention_heads


        resnets = [
            ResnetBlock2D(
                in_channels=in_channels,
                out_channels=in_channels,
                temb_channels=temb_channels,
                eps=resnet_eps,
                groups=resnet_groups,
                dropout=dropout,
                time_embedding_norm=resnet_time_scale_shift,
                non_linearity=resnet_act_fn,
                output_scale_factor=output_scale_factor,
                pre_norm=resnet_pre_norm,
            )
        ]

        cross_attentions = []

        for _ in range(num_layers):
            cross_attentions.append(Transformer2DModel(
                num_attention_heads = num_attention_heads,
                attention_head_dim = in_channels // num_attention_heads,
                in_channels = in_channels,
                num_layers=1,
                cross_attention_dim=cross_attention_dim,
                norm_num_groups=resnet_groups,
                only_cross_attention = False,
                # use_linear_projection=use_linear_projection,
                # upcast_attention=upcast_attention,
            ))

            resnets.append(
                ResnetBlock2D(
                    in_channels=in_channels,
                    out_channels=in_channels,
                    temb_channels=temb_channels,
                    eps=resnet_eps,
                    groups=resnet_groups,
                    dropout=dropout,
                    time_embedding_norm=resnet_time_scale_shift,
                    non_linearity=resnet_act_fn,
                    output_scale_factor=output_scale_factor,
                    pre_norm=resnet_pre_norm,
                )
            )

        self.cross_attentions = nn.ModuleList(cross_attentions)
        self.resnets = nn.ModuleList(resnets)

    def forward(self, hidden_states, temb=None, encoder_hidden_states=None):
        
        hidden_states = self.resnets[0](hidden_states, temb)
        print(hidden_states.shape)
        for attn, resnet in zip(self.cross_attentions, self.resnets[1:]):
            if attn is not None:
                hidden_states = attn(hidden_states,
                                     encoder_hidden_states = encoder_hidden_states,
                                     return_dict=False
                                     )[0]
                hidden_states = resnet(hidden_states, temb)

        return hidden_states

In [18]:
mid = UNetMidBlock2DCrossAttnImageKV(in_channels=64,
                               temb_channels=512,
                               cross_attention_dim=128 
                               )
mid

UNetMidBlock2DCrossAttnImageKV(
  (cross_attentions): ModuleList(
    (0): Transformer2DModel(
      (norm): GroupNorm(32, 64, eps=1e-06, affine=True)
      (proj_in): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (transformer_blocks): ModuleList(
        (0): BasicTransformerBlock(
          (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (attn1): Attention(
            (to_q): Linear(in_features=64, out_features=64, bias=False)
            (to_k): Linear(in_features=64, out_features=64, bias=False)
            (to_v): Linear(in_features=64, out_features=64, bias=False)
            (to_out): ModuleList(
              (0): Linear(in_features=64, out_features=64, bias=True)
              (1): Dropout(p=0.0, inplace=False)
            )
          )
          (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (attn2): Attention(
            (to_q): Linear(in_features=64, out_features=64, bias=False)
            (to_k): Linear(in_featu

In [9]:
temb=torch.randn(3, 512)

In [20]:
feature = torch.randn(3, 64, 32, 32)
fr_emb = torch.randn(3, 32*32, 128)

mid(hidden_states=feature,
    encoder_hidden_states=fr_emb,
    temb=temb)

torch.Size([3, 64, 32, 32])


ValueError: too many values to unpack (expected 3)

In [35]:
feature = torch.randn(3, 64, 32, 32)
fr_emb = torch.randn(3, 64, 32, 32)


mid.cross_attentions[0](feature, fr_emb, return_dict=False)

ValueError: too many values to unpack (expected 3)

In [27]:
feature = torch.randn(3, 64, 16, 16)
fr_emb = torch.randn(3, 64, 32, 32)
temb = torch.randn(3, 512)

In [28]:
mid(hidden_states=feature, temb = temb, cond=fr_emb)

torch.Size([3, 64, 16, 16])


ValueError: too many values to unpack (expected 3)

In [ ]:
UNetMidBlock2DCrossAttn(
    transformer_layers_per_block=transformer_layers_per_block[-1],
    in_channels=block_out_channels[-1],
    temb_channels=blocks_time_embed_dim,
    resnet_eps=norm_eps,
    resnet_act_fn=act_fn,
    output_scale_factor=mid_block_scale_factor,
    resnet_time_scale_shift=resnet_time_scale_shift,
    cross_attention_dim=cross_attention_dim[-1],
    num_attention_heads=num_attention_heads[-1],
    resnet_groups=norm_num_groups,
    dual_cross_attention=dual_cross_attention,
    use_linear_projection=use_linear_projection,
    upcast_attention=upcast_attention,
)